In [ ]:
!pip install kaggle

In [2]:
from google.colab import files

# Subir tu archivo kaggle.json
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c playground-series-s3e20


 67% 33.0M/48.9M [00:00<00:00, 129MB/s]
100% 48.9M/48.9M [00:00<00:00, 150MB/s]


In [ ]:
import zipfile
import os
zip_filename = 'playground-series-s3e20.zip'
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall()
#files = os.listdir()
#files

In [14]:
# Import libraries
import pandas as pd
import numpy as np
import random
import os
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = None

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
d = pd.read_csv("train.csv")
d.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.00,0.60,-0.00,0.26,-98.59,...,3664.44,61085.81,2615.12,15.57,0.27,-12.63,35.63,-138.79,30.75,3.75
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.00,0.73,0.00,0.13,16.59,...,3651.19,66969.48,3174.57,8.69,0.26,30.36,39.56,-145.18,27.25,4.03
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.00,0.75,0.00,0.11,72.80,...,4216.99,60068.89,3516.28,21.10,0.25,15.38,30.40,-142.52,26.19,4.23
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.51,51064.55,4180.97,15.39,0.26,-11.29,24.38,-132.67,28.83,4.31
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.00,0.68,-0.00,0.12,4.12,...,3980.60,63751.13,3355.71,8.11,0.24,38.53,37.39,-141.51,22.20,4.35


Limpiar

In [21]:
columnas_a_eliminar = ['ID_LAT_LON_YEAR_WEEK']
d = d.drop(columnas_a_eliminar, axis=1)
d = d.apply(lambda x: x.fillna(np.random.normal(x.mean(), x.std()) if x.dtype != 'O' else x))
d.to_csv('nuevo_conjunto_de_datos.csv', index=False)

In [22]:
## KEEPOUTPUT
k = d.isna().sum()
k[k!=0]

Series([], dtype: int64)

Separar variables a predecir

In [23]:
X = d.values[:,:-1]
y = d["emission"].values
print (X.shape, y.shape)

(79023, 74) (79023,)


In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error

tomar 30% para test

In [24]:
Xtr, Xts, ytr, yts = train_test_split(X,y, test_size=0.3)
print (Xtr.shape, ytr.shape, Xts.shape, yts.shape)

(55316, 74) (55316,) (23707, 74) (23707,)


Regresion Lineal

In [25]:
lr = LinearRegression()
lr.fit(Xtr, ytr)
lr.score(Xtr, ytr), lr.score(Xts, yts)

(0.029870151924411892, 0.02312901021478353)

validacion cruzada

In [29]:
def rel_mrae(estimator, X, y):
    preds = estimator.predict(X)
    return np.mean(np.abs(preds-y)/y)

In [27]:
from progressbar import progressbar as pbar

def bootstrap_score(estimator, X, y, test_size):
  trscores, tsscores = [], []
  for _ in range(10):
    Xtr, Xts, ytr, yts = train_test_split(X,y, test_size=test_size)
    estimator.fit(Xtr, ytr)
    trscores.append(rel_mrae(estimator, Xtr, ytr))
    tsscores.append(rel_mrae(estimator, Xts, yts))

  return (np.mean(trscores), np.std(trscores)), (np.mean(tsscores), np.std(tsscores))

In [30]:
estimator = LinearRegression()
(trmean, trstd), (tsmean, tsstd) = bootstrap_score(estimator, X, y, test_size=0.3)
print ("train score %.3f (±%.4f)"%(trmean, trstd))
print ("test score  %.3f (±%.4f)"%(tsmean, tsstd))

train score inf (±nan)
test score  inf (±nan)


Bootstrapping

In [31]:
from sklearn.model_selection import ShuffleSplit, KFold,cross_val_score

In [32]:
ss = ShuffleSplit(n_splits=3, test_size=0.3)

for a,b in ss.split(range(10)):
    print (a, b)

[4 2 5 0 9 6 3] [7 8 1]
[4 2 6 9 3 0 5] [7 1 8]
[0 6 7 4 9 3 1] [8 2 5]


In [33]:
z = cross_val_score(lr, X, y, cv = ShuffleSplit(n_splits=10, test_size=0.3), scoring=rel_mrae)
print (z)
print ("test score  %.3f (±%.4f)"%(np.mean(z), np.std(z)))

[inf inf inf inf inf inf inf inf inf inf]
test score  inf (±nan)


In [36]:
ss = KFold(n_splits=3)

for a,b in ss.split(range(10)):
    print (a, b)

[4 5 6 7 8 9] [0 1 2 3]
[0 1 2 3 7 8 9] [4 5 6]
[0 1 2 3 4 5 6] [7 8 9]


In [37]:
z = cross_val_score(lr, X, y, cv = KFold(n_splits=10), scoring=rel_mrae)
print (z)
print ("test score  %.3f (±%.4f)"%(np.mean(z), np.std(z)))

[387.24378881  10.04144557          inf          inf          inf
          inf   5.1812808           inf          inf          inf]
test score  inf (±nan)


random forest

In [50]:
dataset = pd.read_csv("nuevo_conjunto_de_datos.csv")
dataset = dataset.sample(len(dataset))

d = dataset.iloc[:10000].sample(1000)
X = d.values[:,:-1]
y = d["emission"].values
print (X.shape, y.shape)

(1000, 74) (1000,)


In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error

In [44]:
def rel_mrae(estimator, X, y):
    preds = estimator.predict(X)
    return np.mean(np.abs(preds-y)/y)

In [46]:
from sklearn.model_selection import cross_validate, ShuffleSplit

Modelo 1

In [51]:
estimator1 = DecisionTreeRegressor(max_depth=5)
z1 = cross_validate(estimator1, X, y, return_train_score=True, return_estimator=True,
                    scoring=rel_mrae, cv=ShuffleSplit(n_splits=10, test_size=0.1))

In [52]:
z1

{'fit_time': array([0.0490222 , 0.04342651, 0.0400064 , 0.03700352, 0.04074907,
        0.03782821, 0.03822684, 0.03825808, 0.03875065, 0.03477573]),
 'score_time': array([0.00061631, 0.00064945, 0.00062227, 0.00065303, 0.00060606,
        0.00062275, 0.00066495, 0.00068283, 0.00066233, 0.00072742]),
 'estimator': [DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5),
  DecisionTreeRegressor(max_depth=5)],
 'test_score': array([         inf,          inf,          inf,          inf,
        104.60837438,          inf,          inf,          inf,
                 inf,          inf]),
 'train_score': array([nan, nan, inf, inf, inf, inf, inf, inf, inf, inf])}

In [53]:
def report_cv_score(z):
    print ("test score   %.3f (±%.4f) with %d splits"%(np.mean(z["test_score"]), np.std(z["test_score"]), len(z["test_score"])))
    print ("train score  %.3f (±%.4f) with %d splits"%(np.mean(z["train_score"]), np.std(z["train_score"]), len(z["train_score"])))

report_cv_score(z1)

test score   inf (±nan) with 10 splits
train score  nan (±nan) with 10 splits


Modelo 2

In [54]:
from sklearn.ensemble import RandomForestRegressor
estimator2 = DecisionTreeRegressor(max_depth=10)
z2 = cross_validate(estimator2, X, y, return_train_score=True, return_estimator=True,
                    scoring=rel_mrae, cv=ShuffleSplit(n_splits=10, test_size=0.1))
report_cv_score(z2)

test score   inf (±nan) with 10 splits
train score  nan (±nan) with 10 splits


Modelo 3

In [55]:
from sklearn.linear_model import LinearRegression
estimator3 = LinearRegression()
z3 = cross_validate(estimator3, X, y, return_train_score=True, return_estimator=True,
                    scoring=rel_mrae, cv=ShuffleSplit(n_splits=10, test_size=0.1))
report_cv_score(z3)

test score   inf (±nan) with 10 splits
train score  inf (±nan) with 10 splits


Seleccion del modelo

In [57]:
from sklearn.model_selection import train_test_split

test_size = 0.3
number_of_houses_for_trainval = 1000

val_size  = test_size/(1-test_size) # so that the have the same number of elements

assert number_of_houses_for_trainval, "how much emitions"


d = dataset.iloc[:10000].sample(number_of_houses_for_trainval)
X = d.values[:,:-1]
y = d["emission"].values
print (X.shape, y.shape)

print ("test size %.2f"%test_size)
print ("val size is %.2f (relative to %.2f) "%(val_size, 1-test_size))

Xtv, Xts, ytv, yts = train_test_split(X, y, test_size=test_size)
print (Xtv.shape, Xts.shape)

(1000, 74) (1000,)
test size 0.30
val size is 0.43 (relative to 0.70) 
(700, 74) (300, 74)


In [58]:
zscores = []
estimators = [estimator1, estimator2, estimator3]
for estimator in estimators:
    print("--")
    z = cross_validate(estimator, Xtv, ytv, return_train_score=True, return_estimator=False,
                       scoring=rel_mrae, cv=ShuffleSplit(n_splits=10, test_size=val_size))
    report_cv_score(z)
    zscores.append(np.mean(z["test_score"]))
best = np.argmin(zscores)
print ("selecting ", best)
best_estimator = estimators[best]
print ("\nselected model")
print (best_estimator)

--
test score   inf (±nan) with 10 splits
train score  nan (±nan) with 10 splits
--
test score   nan (±nan) with 10 splits
train score  nan (±nan) with 10 splits
--
test score   inf (±nan) with 10 splits
train score  inf (±nan) with 10 splits
selecting  1

selected model
DecisionTreeRegressor(max_depth=10)
